In [1]:
#Importación de librearías necesarias
import pandas as pd
from datetime import datetime
import socket
import numpy as np
import time

In [2]:
#FVariables auxiliares
SBU={ 'ANIO':[2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018],
     'sbu':[105, 122, 136, 150, 160, 170, 200, 218, 240, 264, 292, 318, 340, 354, 366, 375, 386]}

In [3]:
#Path general de archivos
if socket.gethostname()=='LAPTOP-PUSGG08B': #Ip de la laptop
    ruta = "E:/Cristian Guatemal/Master/Big Data y Ciencia de Datos/VIU_TFM/Data/TFM/"
elif socket.gethostname()=='LAPTOP-PUSGG08B11': #Ip del working
    ruta = "E:/Cristian Guatemal/Master/Big Data y Ciencia de Datos/VIU_TFM/Data/TFM/"
# Ruta del archivo de pensionistas de vejez
ruta_vj = ruta + 'POB_VEJ_CD656.dsv'
# Ruta del archivo de historia laboral de pensionistas
ruta_afi = ruta + 'APORTES_CD656.dsv'

In [4]:
vej = pd.read_csv( ruta_vj, delimiter='\t', encoding='utf-8', decimal='.' )
vej['FECHA_NACIMIENTO'] = pd.to_datetime(vej['FECHA_NACIMIENTO'], format='%d/%m/%Y')
vej['FECHA_MUERTE'] = pd.to_datetime(vej['FECHA_MUERTE'], format='%d/%m/%Y')
vej['FECHA_DERECHO'] = pd.to_datetime(vej['FECHA_DERECHO'], format='%d/%m/%Y')
vej['RANGO_INI_5MEJ'] = pd.to_datetime(vej['RANGO_INI_5MEJ'], format='%d/%m/%Y')
vej['RANGO_FIN_5MEJ'] = pd.to_datetime(vej['RANGO_FIN_5MEJ'], format='%d/%m/%Y')

In [5]:
#Datos de vejez
vej.sort_values(by='CEDULA_COD',inplace=True)

In [6]:
vej[vej['CEDULA_COD']==100018]

,CEDULA_COD,SEXO,FECHA_NACIMIENTO,FECHA_MUERTE,FECHA_DERECHO,NUM_IMPOSICIONES,COEFICIENTE_TEO,COEFICIENTE_REAL,BASE_AP_TEO,BASE_AP_REAL,PENSION,TIPO_PRESTACION,RANGO_INI_5MEJ,RANGO_FIN_5MEJ
471930,100018,H,1949-12-08,2020-10-06,2010-03-01,399,0.7875,0.7875,319.68,319.68,498.51,VEJEZ,2019-10-31,2002-03-01


In [7]:
#Se importa el archivo
afi = pd.read_csv(ruta_afi, delimiter='\t', encoding='utf-8', decimal='.')
#afi.sort_values(by='CEDULA_COD',inplace=True)
#afi=afi.set_index(['CEDULA_COD'])
# afi[afi['CEDULA_COD']==114]

In [8]:
afi

,CEDULA_COD,ANIO,MES,SALARIO,PORCENTAJE_AP,SECTOR
0,20057738,2003,3,135.03,20.5,PRIVADO
1,15613923,2003,11,65.61,20.5,PRIVADO
2,16057398,1996,11,56.65,20.5,PRIVADO
3,16023482,1997,9,15.11,20.5,PRIVADO
4,16021285,1998,1,27.27,20.5,PUBLICO
...,...,...,...,...,...,...
66555199,16773012,2005,9,230.00,20.5,PRIVADO
66555200,5709849,2007,1,278.95,30.5,PUBLICO
66555201,19960695,2008,9,1400.00,20.5,PRIVADO
66555202,11118795,1993,3,56.65,20.5,PRIVADO


In [ ]:
afi['SECTOR'].unique()

In [9]:
#Cedulas y aparición en cada sector
ced_sec = afi.groupby('CEDULA_COD')['SECTOR'].nunique()

In [ ]:
ced_sec[ced_sec==2]

In [ ]:
afi[afi['CEDULA_COD']=='100018']['SECTOR'].unique()

In [10]:
#Cédulas que tiene un único sector
uni_ced_sec = ced_sec[ced_sec == 1].index.tolist()
#Cédulas que tiene un dos sector
dos_ced_sec = ced_sec[ced_sec == 2].index.tolist()
#Cédulas que tiene un tres sector
tres_ced_sec = ced_sec[ced_sec == 3].index.tolist()
#Cédulas que tiene más tres sector
mas_ced_sec = ced_sec[ced_sec > 3].index.tolist()

In [11]:
#Afliados de HL con un único sector
afi_uni = afi[afi['CEDULA_COD'].isin(uni_ced_sec)]
#Afliados de HL con un único sector
afi_dos = afi[afi['CEDULA_COD'].isin(dos_ced_sec)]
#Afliados de HL con un único sector
afi_tres = afi[afi['CEDULA_COD'].isin(tres_ced_sec)]
#Afliados de HL con un único sector
afi_mas = afi[afi['CEDULA_COD'].isin(mas_ced_sec)]

In [12]:
#Se presenta la clasificación
print('Se tiene un total ',afi['CEDULA_COD'].nunique(), ' personas que se va analizar su HL')
print('Se tiene un total ',afi_uni['CEDULA_COD'].nunique(), ' personas que tienen un único sector')
print('Se tiene un total ',afi_dos['CEDULA_COD'].nunique(), ' personas que tienen dos sectores')
print('Se tiene un total ',afi_tres['CEDULA_COD'].nunique(), ' personas que tienen tres sectores')
print('Se tiene un total ',afi_mas['CEDULA_COD'].nunique(), ' personas que tienen más de tres sectores')

Se tiene un total  556782  personas que se va analizar su HL
Se tiene un total  391458  personas que tienen un único sector
Se tiene un total  136187  personas que tienen dos sectores
Se tiene un total  26115  personas que tienen tres sectores
Se tiene un total  3022  personas que tienen más de tres sectores


In [ ]:
391458+136187+26115+3022

In [ ]:
#Primero se va a trabajar con las cedúlas que tiene un único sector
afi_uni['SECTOR'].unique()

In [ ]:
afi_uni[afi_uni['CEDULA_COD']=='16021285']['SECTOR'].unique()

In [103]:
#Se desagrea por sector 
pri=afi_uni[afi_uni['SECTOR']=='PRIVADO']
#pri.sort_values(by='CEDULA_COD',inplace=True)
pub=afi_uni[afi_uni['SECTOR']=='PUBLICO']
#pub.sort_values(by='CEDULA_COD',inplace=True)
ind=afi_uni[afi_uni['SECTOR']=='INDEPENDIENTES']
#ind.sort_values(by='CEDULA_COD',inplace=True)
# vol_ext=afi[(afi['SECTOR']=='69-VOLUNTARIO ECUATORIANO DOMICILIADO EN EL EXTERIOR ') |
#             afi['SECTOR']=='89-AFILIACION DOMICILIADO EN EL EXTERIOR']
# vol_ecu=afi[afi['SECTOR']=='90-AFILIACION DOMICILIADO EN EL PAIS / SIN RELACION DEPENDENCIA']

In [104]:
pri=pri.sort_values( by=["CEDULA_COD", "ANIO", "MES"], ascending=[True, False, False] )

In [186]:
#Función para crear los grupos de salarios de 12 meses consecutivos para cada cedula
def asignar_grupo(df, n_grupo):
    df['GRUPO'] = (df.groupby('CEDULA_COD').cumcount() // n_grupo ) + 1
    df['SAL_PROM_GRUPO'] = df.groupby(['CEDULA_COD', 'GRUPO'])['SALARIO'].transform('mean')
    return df

#Función para agrupar los valores de los sueldos
def valores_unicos_grupos(df):
    df1= df.groupby(['CEDULA_COD', 'GRUPO'])['SAL_PROM_GRUPO'].unique()
    df1= df1.reset_index()
    df1['SAL_PROM_GRUPO']=df1['SAL_PROM_GRUPO'].apply(lambda x: x[0])
    df1=df1.sort_values(by=['CEDULA_COD','SAL_PROM_GRUPO'], ascending=[True,False])
    return df1

#Función para seleccionar los 5 mejores años de sueldo promedio
def sel_top_salarios( df, top ):
    top_sal = df.groupby('CEDULA_COD')['SAL_PROM_GRUPO'].nlargest(top).reset_index(level=0, drop=True)
    indices_top3 = top_sal.index
    df1 = df.loc[indices_top3].reset_index(drop=True)
    return df1

#Función para seleccionar los grupos de sueldos
def sel_grupo(df1, df2):
    combinaciones = set(zip(df2['CEDULA_COD'], df2['GRUPO']))
    df1['SEL_GRUPO'] = [(cedula, grupo) in combinaciones for cedula, grupo in zip(df1['CEDULA_COD'], df1['GRUPO'])]
    df1['SEL_GRUPO'] = df1['SEL_GRUPO'].astype(int)
    return df1

#Función para calcular la base de calculo teórica
def base_cal(df):
    grupo = df[df['SEL_GRUPO'] == 1]
    df_prom = grupo.groupby('CEDULA_COD')['SALARIO'].mean()
    df['BASE_CAL'] = df['CEDULA_COD'].map( df_prom )
    return df

#Función para calcular los valores atipicos dentro de los grupos seleccionados
def ati_base_cal(df):
    grupo = df[df['SEL_GRUPO'] == 1]
    Q1= grupo.groupby('CEDULA_COD')['SALARIO'].quantile(0.25)
    Q3= grupo.groupby('CEDULA_COD')['SALARIO'].quantile(0.75)
    IQR = Q3-Q1
    LI = Q1 - 1.5 * IQR
    LS = Q3 + 1.5 * IQR
    df['IQR'] = df['CEDULA_COD'].map( IQR )
    df['LI'] = df['CEDULA_COD'].map( LI )
    df['LS'] = df['CEDULA_COD'].map( LS )
    df['ATI'] = ((df['SALARIO'] < df['LI']) | (df['SALARIO'] > df['LS'])).astype(int)
    
    grupo = df[df['SEL_GRUPO'] == 1]
    n_ati = ( grupo.groupby('CEDULA_COD')['ATI'].sum()/grupo.groupby('CEDULA_COD')['ATI'].count() ) * 100
    df['POR_ATI'] = df['CEDULA_COD'].map( n_ati )
    return df

#Función para calcular la base de calculo sin los valores atipicos dentro de los grupos seleccionados
def base_cal_sin_ati(df):
    grupo = df[ (df['SEL_GRUPO'] == 1 ) & (df['ATI'] != 1)]
    df_prom = grupo.groupby('CEDULA_COD')['SALARIO'].mean()
    df['BASE_CAL_SIN_ATI'] = df['CEDULA_COD'].map( df_prom )
    return df

In [115]:
n_grupo = 12 # Grupos de 12
df = asignar_grupo(pri, n_grupo )
df1 = valores_unicos_grupos(df)
top = 5 # 5 Mejores años de sueldo
df2 = sel_top_salarios(df1,top)
df3 = sel_grupo(df, df2)
df4 = base_cal(df3)
df5 = ati_base_cal(df4)

In [192]:
#df5 = ati_base_cal(df4)
#df5 = base_cal_sin_ati(df5)
#df3[df3['CEDULA_COD']==100018]
#df4[df4['CEDULA_COD']==100018]['SALARIO'].quantile(0.25)
df5[df5['POR_ATI']>5]

,CEDULA_COD,ANIO,MES,SALARIO,PORCENTAJE_AP,SECTOR,GRUPO,SALARIO_SUMADO,SEL_GRUPO,SALARIO_MEDIO,Base_Cal,IQR,LI,LS,ATI,BASE_CAL_SIN_ATI,POR_ATI
1089807,115,2006,4,11.67,20.5,PRIVADO,1,312.407778,1,312.407778,312.407778,0.0000,350.00000,350.00000,1,350.0000,11.111111
15464706,115,2006,3,350.00,20.5,PRIVADO,1,312.407778,1,312.407778,312.407778,0.0000,350.00000,350.00000,0,350.0000,11.111111
6217219,115,2006,2,350.00,20.5,PRIVADO,1,312.407778,1,312.407778,312.407778,0.0000,350.00000,350.00000,0,350.0000,11.111111
28832615,115,2006,1,350.00,20.5,PRIVADO,1,312.407778,1,312.407778,312.407778,0.0000,350.00000,350.00000,0,350.0000,11.111111
29577534,115,2005,12,350.00,20.5,PRIVADO,1,312.407778,1,312.407778,312.407778,0.0000,350.00000,350.00000,0,350.0000,11.111111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7013457,23599026,2008,7,3129.78,41.0,PRIVADO,1,2764.713333,1,2764.713333,2764.713333,158.9925,2877.78375,3513.75375,0,3177.8525,33.333333
12422697,23599026,2008,6,3154.26,41.0,PRIVADO,1,2764.713333,1,2764.713333,2764.713333,158.9925,2877.78375,3513.75375,0,3177.8525,33.333333
48551687,23599026,2008,5,3315.60,41.0,PRIVADO,1,2764.713333,1,2764.713333,2764.713333,158.9925,2877.78375,3513.75375,0,3177.8525,33.333333
54278199,23599026,2008,4,3111.77,41.0,PRIVADO,1,2764.713333,1,2764.713333,2764.713333,158.9925,2877.78375,3513.75375,0,3177.8525,33.333333


In [193]:
6228052/19758517

0.31520847440119115

In [188]:
#df5.groupby('CEDULA_COD')['ATI'].count()
#df5[ (df5['CEDULA_COD']==100018)]
df5[ (df5['CEDULA_COD']==100018) & (df5['SEL_GRUPO']==1)]

#df5[(df5['CEDULA_COD']==100018) & (df5['SEL_GRUPO']==1)]['ATI'].count()

,CEDULA_COD,ANIO,MES,SALARIO,PORCENTAJE_AP,SECTOR,GRUPO,SALARIO_SUMADO,SEL_GRUPO,SALARIO_MEDIO,Base_Cal,IQR,LI,LS,ATI,BASE_CAL_SIN_ATI,POR_ATI
53562255,100018,2019,10,266.00,20.6,PRIVADO,1,422.959167,1,422.959167,406.147833,46.3475,311.47875,496.86875,1,404.370351,5.0
26525488,100018,2019,9,416.46,41.2,PRIVADO,1,422.959167,1,422.959167,406.147833,46.3475,311.47875,496.86875,0,404.370351,5.0
49624029,100018,2019,8,463.84,41.2,PRIVADO,1,422.959167,1,422.959167,406.147833,46.3475,311.47875,496.86875,0,404.370351,5.0
19896422,100018,2019,7,399.00,20.6,PRIVADO,1,422.959167,1,422.959167,406.147833,46.3475,311.47875,496.86875,0,404.370351,5.0
35162829,100018,2019,6,403.99,41.2,PRIVADO,1,422.959167,1,422.959167,406.147833,46.3475,311.47875,496.86875,0,404.370351,5.0
18279480,100018,2019,5,399.00,20.6,PRIVADO,1,422.959167,1,422.959167,406.147833,46.3475,311.47875,496.86875,0,404.370351,5.0
58524527,100018,2019,4,399.00,20.6,PRIVADO,1,422.959167,1,422.959167,406.147833,46.3475,311.47875,496.86875,0,404.370351,5.0
53068659,100018,2019,3,512.47,41.2,PRIVADO,1,422.959167,1,422.959167,406.147833,46.3475,311.47875,496.86875,1,404.370351,5.0
52598696,100018,2019,2,415.21,41.2,PRIVADO,1,422.959167,1,422.959167,406.147833,46.3475,311.47875,496.86875,0,404.370351,5.0
9516049,100018,2019,1,440.15,41.2,PRIVADO,1,422.959167,1,422.959167,406.147833,46.3475,311.47875,496.86875,0,404.370351,5.0


In [174]:
(404.370351/406.147833 -1)*100

-0.43764409300688367

In [ ]:
#Sector privado
inicio = time.time()
pri_t = pri.pivot_table(index=('CEDULA_COD'), columns=('ANIO', 'MES'), values=('SALARIO'),aggfunc=lambda x: x)
pri_t.sort_values(by='CEDULA_COD',inplace=True)
fin = time.time()
print('Tiempo de ejecución:\n')
print('horas', (fin-inicio)//3600 )
print('minutos', ((fin-inicio)%3600)//60 )
print('segundos', (fin-inicio)%60 )

In [ ]:
pri_t

In [ ]:
pri_t['Bas_Cal'] = pd.NA
pri_t['Fec_Ini'] = pd.NA
pri_t['Fec_Fin'] = pd.NA

In [ ]:
inicio = time.time()

meses = 12
base = 5

for persona in pri_t.index[:10000]:
    
    col_sel=pri_t.loc[persona].notna()
    columnas_sin_nan = pri_t.columns[col_sel & ( pri_t.columns != "CEDULA_COD")  & ( pri_t.columns != "Bas_Cal") & ( pri_t.columns != "Fec_Ini") & ( pri_t.columns != "Fec_Fin")  ][::-1]
    grupos = [columnas_sin_nan[i:i+meses] for i in range(0, len(columnas_sin_nan), meses)]
    
    sumas_por_grupo = {}
    for grupo in grupos:
        suma_grupo = pri_t[grupo].mean(axis=1).values[0]
        sumas_por_grupo[tuple(grupo)] = suma_grupo
        # Ordenar los grupos por su suma en orden descendente
        grupos_ordenados = sorted(sumas_por_grupo.items(), key=lambda x: x[1], reverse=True)[:base]
    
    componentes1 = [[par for par in tupla[0]] for tupla in grupos_ordenados]
    componentes = [item for sublist in componentes1 for item in sublist]
    
    clave = lambda tupla: tupla[0] * 100 + tupla[1]
    # Encontrar el par de año y mes mínimo y máximo usando la clave personalizada
    par_minimo = min((min(sublista, key=clave ) for sublista in componentes1), key=clave )
    par_maximo = max((max(sublista, key=clave ) for sublista in componentes1), key=clave )
    
    pri_t.loc[pri_t.index==persona,'Bas_Cal'] = pri_t.iloc[pri_t.index==persona][componentes].mean(axis=1)
    pri_t.loc[pri_t.index==persona,'Fec_Ini'] = "{}/{}".format(par_maximo[0], str(par_maximo[1]).zfill(2))
    pri_t.loc[pri_t.index==persona,'Fec_Fin'] = "{}/{}".format(par_minimo[0], str(par_minimo[1]).zfill(2))
        
fin = time.time()
print('Tiempo de ejecución:\n')
print('horas', (fin-inicio)//3600 )
print('minutos', ((fin-inicio)%3600)//60 )
print('segundos', (fin-inicio)%60 )   

In [ ]:
pri_t

In [ ]:
col_sel=pri.loc[persona].notna()
columnas_sin_nan = pri.columns[col_sel][::-1]
columnas_sin_nan
# grupos = [columnas_sin_nan[i:i+meses] for i in range(0, len(columnas_sin_nan), meses)]
    
# sumas_por_grupo = {}
# for grupo in grupos:
#     suma_grupo = pri[grupo].mean(axis=1).values[0]
#     sumas_por_grupo[tuple(grupo)] = suma_grupo
#         # Ordenar los grupos por su suma en orden descendente
#     grupos_ordenados = sorted(sumas_por_grupo.items(), key=lambda x: x[1], reverse=True)[:base]
    
# componentes1 = [[par for par in tupla[0]] for tupla in grupos_ordenados]
# componentes = [item for sublist in componentes1 for item in sublist]
    
# clave = lambda tupla: tupla[0] * 100 + tupla[1]
#     # Encontrar el par de año y mes mínimo y máximo usando la clave personalizada
# par_minimo = min((min(sublista, key=clave ) for sublista in componentes1), key=clave )
# par_maximo = max((max(sublista, key=clave ) for sublista in componentes1), key=clave )
    
# pri.loc[pri.index==persona,'Base_calculada'] = pri.iloc[pri.index==persona][componentes].mean(axis=1)
# pri.loc[pri.index==persona,'Fec_Ini'] = pd.to_datetime('{},{}'.format( par_maximo[0], par_maximo[1]), format='%Y, %m')
# pri.loc[pri.index==persona,'Fec_Fin'] = pd.to_datetime('{},{}'.format( par_minimo[0], par_minimo[1]), format='%Y, %m')

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import mean
from pyspark.sql.functions import col
from pyspark.sql.functions import lit
from pyspark.sql.functions import concat_ws
from pyspark.sql.types import IntegerType

spark = SparkSession.builder \
    .appName("YourApp") \
    .getOrCreate()

# Definir meses y base
meses = 12
base = 5



In [ ]:
pri_spark = spark.createDataFrame(pd.DataFrame(pri))
pri_spark

In [ ]:
#Para determinar las columnas con valores diferentes a NaN
col_sel=pri.loc['100018'].notna()
columnas_sin_nan = pri.columns[col_sel][::-1]

In [ ]:
meses = 12
grupos = [columnas_sin_nan[i:i+meses] for i in range(0, len(columnas_sin_nan), meses)]

In [ ]:
pri.iloc[pri.index=='100018'].stack(0)

In [ ]:
440.15+415.21+512.47+399.00+399.00+403.99+399.00+463.84+416.46+266.00+541.29+419.10

In [ ]:
base = 5
sumas_por_grupo = {}
for grupo in grupos:
    suma_grupo = pri[grupo].mean(axis=1).values[0]
    sumas_por_grupo[tuple(grupo)] = suma_grupo

# Ordenar los grupos por su suma en orden descendente
grupos_ordenados = sorted(sumas_por_grupo.items(), key=lambda x: x[1], reverse=True)[:base]
#grupos_ordenados

In [ ]:
componentes1 = [[par for par in tupla[0]] for tupla in grupos_ordenados]
componentes = [item for sublist in componentes1 for item in sublist]

clave_personalizada = lambda tupla: tupla[0] * 100 + tupla[1]

# Encontrar el par de año y mes mínimo y máximo usando la clave personalizada
par_minimo = min((min(sublista, key=clave_personalizada) for sublista in componentes1), key=clave_personalizada)
par_maximo = max((max(sublista, key=clave_personalizada) for sublista in componentes1), key=clave_personalizada)

In [ ]:
pri['Base_calculada']=pd.NA
pri['Fec_Ini'] = pd.NA
pri['Fec_Fin'] = pd.NA

In [ ]:
pri['Base_calculada'] = pri.iloc[pri.index=='100018'][componentes].mean(axis=1)
pri.loc[pri.index=='100018','Fec_Ini'] = pd.to_datetime('{}, {}'.format( par_maximo[0], par_maximo[1]), format='%Y, %m')
pri.loc[pri.index=='100018','Fec_Fin'] = pd.to_datetime('{}, {}'.format( par_minimo[0], par_minimo[1]), format='%Y, %m')

In [ ]:
pri

In [ ]:
vej[vej['CEDULA_COD']==114]

In [ ]:
afi[afi['CEDULA_COD']==114]

In [ ]:
afi.loc[['114']]

In [ ]:
import pickle

# Nombre del archivo donde se guardará el objeto
nombre_archivo = 'proceso.pkl'

# Guardar el objeto en el archivo
with open(nombre_archivo, 'wb') as archivo:
    pickle.dump(pri, archivo)

In [ ]:
import pickle

# Nombre del archivo donde se guardó el objeto
nombre_archivo = 'proceso.pkl'

# Cargar el objeto desde el archivo
with open(nombre_archivo, 'rb') as archivo:
    mi_objeto_cargado = pickle.load(archivo)

print(mi_objeto_cargado)